<a href="https://colab.research.google.com/github/zakilbaki/ml-2025/blob/main/02_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path
IMAGES_PATH = Path() / "images" / "classification"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

MNIST = Modified National Institute of Standards and Technology database.

C’est un jeu de données hyper classique de chiffres manuscrits.

Contenu :

70 000 images

Chaque image = 28x28 pixels ➔ soit 784 features (28x28 = 784).

Les valeurs sont en niveaux de gris (0-255).

Chaque image est une représentation d’un chiffre entre 0 et 9.



ce dataset a été crée par yann le cun qui est un recheur francais qui travail en ia qui le chief ia scientiest chez meta et c'est lui qui a popularisé les cnn  et c'est lui crée lenet
Le premier vrai "succès industriel" de LeNet = lire automatiquement les montants sur les chèques de banque aux USA.

In [3]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [ ]:
mnist.keys()  # extra code – we only use data and target in this notebook


In [4]:
X, y = mnist.data, mnist.target

Pour pouvoir utiliser scikit-learn et les modèles ML classiques, tu dois aplatir tes images en vecteurs.




In [ ]:
import matplotlib.pyplot as plt

def plot_digit(image_data):
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

some_digit = X[0]
plot_digit(some_digit)
save_fig("some_digit_plot")  # extra code
plt.show()

ce code permet de reconstruire une image à partir de sa forme matricielle


In [ ]:
# extra code – this cell generates and saves Figure 3–2
plt.figure(figsize=(9, 9))
for idx, image_data in enumerate(X[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)
plt.subplots_adjust(wspace=0, hspace=0)
save_fig("more_digits_plot", tight_layout=False)
plt.show()

le slicing est tres important ici , il a permis de recuperer les 100 premières lignes de X


In [5]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


on peut facilement faire ca avec train_test_split


# training a binary classifier


In [ ]:
y_train_5 = (y_train == '5')  # True for all 5s, False for all other digits
y_test_5 = (y_test == '5')

si y_train==5 => True else false
c'est ceux dont on a besoin pour notre classification binaire


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

on fixe la seed encore à 42 = valeur par défaut en datascience
## pour une classification binaire il faut que la target soit binaire aussi
# ici on utilise le stochastic gradient descent classifier

on apprentissage supérvisé on cherche à mimiser une fonction cout
on a alors la deux option :
##LogisticRegression() = modèle linéaire + solveur optimisé
➔ très précis, bon pour des datasets moyens ou petits.

##SGDClassifier() = modèle linéaire + entraînement par descente de gradient stochastique
➔ rapide et scalable sur gros datasets.
du coup l'approche stochastique est probabiliste



In [ ]:
sgd_clf.predict([some_digit])


le modéle prédit bien
On sépare toujours nos données :
Train set : pour entraîner le modèle

Test set : pour évaluer la capacité à généraliser

(parfois aussi un Validation set pour régler les hyperparamètres)



# *testons maintenant les performances du modèles *



In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

le but de ce code est de voir la capacité d'apprentissage du modéle en utilisant la validations croisé

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3)  # add shuffle=True if the dataset is not
                                       # already shuffled
for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

ce code montre comment on peut faire la validarion croisee  
StratifiedKFold(n_splits=3)
aprés boucle sur une méthode de l'objet skfold

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier()
dummy_clf.fit(X_train, y_train_5)
print(any(dummy_clf.predict(X_train)))

## **ici on utilise le dummy classifier pour tester notre modéle , celui ci predit toujours le resultat le plus fréquent**
# l'intéret est de voir si le modéle a vraiment appris qlq chose ou si il est inutile

In [ ]:
cross_val_score(dummy_clf, X_train, y_train_5, cv=3, scoring="accuracy")


# accuracy_dummy< accuracy_sgd du coup le modèle a bien appris qlq chose


#  ⚠ le dummy classifier peut etre utiliser sur la classification multidimontionnel

condusion matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

Très intéressent prediction basé sur le meme principe de la  cross_validation j'aime l'idée

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5, y_train_pred)
cm

la confusion marix contien les TN ET LES TF sur les diagonales et les FN FP

L’accuracy seule peut cacher des problèmes (surtout en dataset déséquilibré).

Avec la confusion matrix ➔ tu vois si :

Ton modèle détecte bien les 5 (TP / FN).

Ton modèle fait beaucoup de fausses alarmes (FP).

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)  # == 3530 / (687 + 3530)= TP/TP+FP 	Parmi les "5" que j’ai prédit, combien étaient vraiment des "5"

PROBALISTIQUEMENT : ca donne le pourcentage de reuissite si un 5 est predit



In [ ]:
recall_score(y_train_5, y_train_pred)  # == 3530 / (1891 + 3530) (TP/TP+FN) ( Parmi tous les vrais "5", combien j’en ai correctement détecté  )


Problème	On privilégie quoi ?
Détection de cancer	Recall
Filtrage anti-spam	Precision
Reconnaissance d’empreintes	Precision
Reconnaissance de visages en sécurité	Recall

afin de mieux comprendre pourquoi rappele toi que pour le cancer on utilise le recall et pour le filtrage anti-spam on utlilise la précision  


In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

ici on voit le f1_score= 2 (r*p/r+p) il permet de mesurer les 2 cad à qu'elle point le modele est bon dans la precision et le recall


Suis-je globalement bon dans les deux ?

In [ ]:
#recall et tradeoff
y_scores = sgd_clf.decision_function([some_digit])
y_scores

Ça renvoie le score de distance par rapport à l'hyperplan de décision.
Plus le score est grand ➔ plus le modèle est confiant que c’est un "5".
Plus le score est négatif ➔ plus il est sûr que ce n’est pas un "5"

Le cœur de presque tous les modèles linéaires (et même beaucoup de non-linéaires) repose sur cette fameuse expression :𝑤⋅x+𝑏
ce score est la distance entre la prediction et l'hyperplan de décision

In [ ]:
threshold = 0

y_some_digit_pred = (y_scores > threshold)


In [ ]:
y_some_digit_pred
#il faut remarquer que pour un threshold égale à 0 le resultat est le méme qu'appeler la fonction predict


In [ ]:
threshold = 3000
y_some_digit_pred = (y_scores > threshold)
y_some_digit_pred

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")
#ici le code va retourner les sortie de la decision function d(x)=wx+b


l'intéret de ca et pour après càd que ca permettra de choisir le threashold optimal pour toute precision voulu

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
plt.figure(figsize=(8, 4))  # extra code – it's not needed, just formatting
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")

# extra code – this section just beautifies and saves Figure 3–5
idx = (thresholds >= threshold).argmax()  # first index ≥ threshold
plt.plot(thresholds[idx], precisions[idx], "bo")
plt.plot(thresholds[idx], recalls[idx], "go")
plt.axis([-50000, 50000, 0, 1])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")
save_fig("precision_recall_vs_threshold_plot")

plt.show()

Le seuil optimal ne sort pas magiquement du modèle.
Tu dois l’ajuster en fonction de ce que tu veux (médecine, finance, sécurité...).

In [ ]:
import matplotlib.patches as patches  # extra code – for the curved arrow

plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting

plt.plot(recalls, precisions, linewidth=2, label="Precision/Recall curve")

# extra code – just beautifies and saves Figure 3–6
plt.plot([recalls[idx], recalls[idx]], [0., precisions[idx]], "k:")
plt.plot([0.0, recalls[idx]], [precisions[idx], precisions[idx]], "k:")
plt.plot([recalls[idx]], [precisions[idx]], "ko",
         label="Point at threshold 3,000")
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.79, 0.60), (0.61, 0.78),
    connectionstyle="arc3,rad=.2",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.56, 0.62, "Higher\nthreshold", color="#333333")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.axis([0, 1, 0, 1])
plt.grid()
plt.legend(loc="lower left")
save_fig("precision_vs_recall_plot")

plt.show()

ce code met en évidence le threshold optimal pour avoir 90% de precision correspond à 3000

In [ ]:
idx_for_90_precision = (precisions >= 0.90).argmax()
## ici on cherche le premier seuil
threshold_for_90_precision = thresholds[idx_for_90_precision]
threshold_for_90_precision

In [ ]:
y_train_pred_90 = (y_scores >= threshold_for_90_precision) #true id y_scores > Th else falsed
precision_score(y_train_5, y_train_pred_90)


In [ ]:
import numpy as np
# on filtre les seuils qui respectent la contrainte
valid = np.where(recalls >= 0.90)[0]
# on prend le plus grand seuil parmi ceux qui respectent la contrainte du coup le argmax ne marche pas
best_idx = valid[-1]  # dernier élément compatible
threshold_for_90_recall = thresholds[best_idx]
threshold_for_90_recall
y_train_pred_90r = (y_scores >= threshold_for_90_recall) #true id y_scores > Th else falsed
recall_score(y_train_5, y_train_pred_90r)



# THE ROC CURVE

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

idx_for_threshold_at_90 = (thresholds <= threshold_for_90_precision).argmax()
tpr_90, fpr_90 = tpr[idx_for_threshold_at_90], fpr[idx_for_threshold_at_90]

plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting
plt.plot(fpr, tpr, linewidth=2, label="ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.plot([fpr_90], [tpr_90], "ko", label="Threshold for 90% precision")

# extra code – just beautifies and saves Figure 3–7
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.20, 0.89), (0.07, 0.70),
    connectionstyle="arc3,rad=.4",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.12, 0.71, "Higher\nthreshold", color="#333333")
plt.xlabel('False Positive Rate (Fall-Out)')
plt.ylabel('True Positive Rate (Recall)')
plt.grid()
plt.axis([0, 1, 0, 1])
plt.legend(loc="lower right", fontsize=13)
save_fig("roc_curve_plot")

plt.show()

precision_recall Très bon pour datasets déséquilibrés (comme ton MNIST 5 ou pas 5)
ROC	Bon pour évaluer globalement la capacité de discrimination du modèle


In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

Étape	Ce que tu fais
1. Entraînement	Tu entraînes 5 modèles
2. Évaluation AUC	Tu calcules roc_auc_score() sur val set
3. Sélection modèle	Tu gardes celui avec meilleur AUC
4. Calibration de seuil	Tu fais precision_recall_curve() et choisis ton seuil métier
5. Déploiement	Tu pars en production avec ton modèle calibré

AUC ROC = vision globale "mon modèle voit-il quelque chose ?"
Precision-Recall = vision fine "comment il se comporte dans mon métier concret ?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)

In [ ]:
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

In [ ]:
y_probas_forest[:3]


In [ ]:
idx_50_to_60 = (y_probas_forest[:, 1] > 0.50) & (y_probas_forest[:, 1] < 0.60)
print(f"{(y_train_5[idx_50_to_60]).sum() / idx_50_to_60.sum():.1%}")

➔ La calibration n’est utile que quand tu as besoin que les probabilités soient fiables.
Un modèle calibré = ses probabilités sont "honnêtes" et reflètent la vraie fréquence d’occurrence.



pipeline d'évaluation

In [ ]:
y_scores_forest = y_probas_forest[:, 1]
precisions_forest, recalls_forest, thresholds_forest = precision_recall_curve(
    y_train_5, y_scores_forest)


In [ ]:
plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting

plt.plot(recalls_forest, precisions_forest, "b-", linewidth=2,
         label="Random Forest")
plt.plot(recalls, precisions, "--", linewidth=2, label="SGD")

# extra code – just beautifies and saves Figure 3–8
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.axis([0, 1, 0, 1])
plt.grid()
plt.legend(loc="lower left")
save_fig("pr_curve_comparison_plot")

plt.show()

In [ ]:
y_train_pred_forest = y_probas_forest[:, 1] >= 0.5  # positive proba ≥ 50%
f1_score(y_train_5, y_train_pred_forest)

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)


In [ ]:
precision_score(y_train_5, y_train_pred_forest)


In [ ]:
recall_score(y_train_5, y_train_pred_forest)

# Part_2


classification multi


the svm scales hardly on large dataset's that's why we will train it on 2000 samlples

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(random_state=42)
svm_clf.fit(X_train[:2000], y_train[:2000])

the svm draws the best line that gives the best marge
marge = distance between the class and the hyperplan
the closest points to the barriere define the support  vector
the other points the svm dosen't see them

if we can't trace a perfect line we introduce an error parametre c
Grand C → moins d’erreurs autorisées → marge plus fine
Petit C → plus d’erreurs autorisées → marge plus large → modèle plus généraliste





5. SVM non linéaire —
le Kernel Trick
Quand les données ne sont pas linéairement séparables (ex: spirale, cercles...), le SVM utilise des noyaux (kernels).
L’idée : projeter les données dans un espace de dimension plus haute où elles deviennent linéairement séparables.

linear → simple hyperplan

rbf (radial basis function) → pour courbes complexes

poly → polynômial



⚠️ Attention :
Plus l’espace est complexe, plus le risque de surapprentissage est élevé (→ d’où l’intérêt de régulariser avec C et parfois gamma).
Plus le kernel est complexe, plus l'entraînement est lent (surtout avec beaucoup de données).

In [ ]:
some_digit_scores = svm_clf.decision_function([some_digit])
some_digit_scores.round(2)

In [ ]:
class_id = some_digit_scores.argmax()
class_id

the svm uses the one v one strategy it's like a league in multiclass

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

ovr_clf = OneVsRestClassifier(SVC(random_state=42))
ovr_clf.fit(X_train[:2000], y_train[:2000])

it's a bit different than the ovo model how compare each two classes , this one compare one vs all the othere = 5 or not 5 so in this case you'll get 10 binary classifiers

in summary the svm just draws the lines then you schould the way you wanna do multiclass classification

In [ ]:
ovr_clf.predict([some_digit])

svc uses ovo by default
linearsvc uses ovr by default

In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype("float64"))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

Scalling is imported because the grandient descent based models the weights converge rapidly so it can be betwenn 0-1 and another feature between 0-1000

🔑 Critères de choix
Taille du jeu de données

Petit (< 10 000 échantillons) → SVM, logistic, Naïve Bayes, k-NN

Moyen (10 000–1 M) → Random Forest, Gradient Boosting, SVM linéaire (LinearSVC)

Grand (> 1 M) → SGDClassifier, réseaux de neurones, LightGBM

Complexité de la frontière

Linéaire → Logistic, LinearSVC, SGDClassifier

Non linéaire → SVC (kernel), Random Forest, Gradient Boosting

Bruit & valeurs manquantes

Robustes → arbres (RF, GB), Naïve Bayes

Sensibles → SVM, k-NN, réseaux de neurones (nécessitent preprocessing)

Exigences métier

Interprétabilité → Logistic, arbres (avec export de règles), SHAP sur GB

Temps réel / streaming → SGDClassifier, Perceptron

Probas fiables → Logistic, calibrage (CalibratedClassifierCV sur RF/SVM)


# ANALYSE D'ERREUR

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
plt.rc('font', size=9)  # extra code – make the text smaller
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred)
plt.show()

matrices très intéressente , elle montre comment le modéle se trompe et qu'elle classes il conffond

In [ ]:
plt.rc('font', size=10)  # extra code
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred,
                                        normalize="true", values_format=".0%")
plt.show()

in this cell the goal is to to see the recall (it's the diagonals) so  but in a different way than the binary you can say it gives the probability of confusing the true number with another number

In [ ]:
sample_weight = (y_train_pred != y_train)
plt.rc('font', size=10)  # extra code
ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred,
                                        sample_weight=sample_weight,
                                        normalize="true", values_format=".0%")
plt.show()

this code choose if we had a false prediction wich are the probalies there another number to see with what he confused
PS here the new parametre is the sample weight

In [ ]:
cl_a, cl_b = '3', '5'
X_aa = X_train[(y_train == cl_a) & (y_train_pred == cl_a)]
X_ab = X_train[(y_train == cl_a) & (y_train_pred == cl_b)]
X_ba = X_train[(y_train == cl_b) & (y_train_pred == cl_a)]
X_bb = X_train[(y_train == cl_b) & (y_train_pred == cl_b)]

In [ ]:
# extra code – this cell generates and saves Figure 3–11
size = 5
pad = 0.2
plt.figure(figsize=(size, size))
for images, (label_col, label_row) in [(X_ba, (0, 0)), (X_bb, (1, 0)),
                                       (X_aa, (0, 1)), (X_ab, (1, 1))]:
    for idx, image_data in enumerate(images[:size*size]):
        x = idx % size + label_col * (size + pad)
        y = idx // size + label_row * (size + pad)
        plt.imshow(image_data.reshape(28, 28), cmap="binary",
                   extent=(x, x + 1, y, y + 1))
plt.xticks([size / 2, size + pad + size / 2], [str(cl_a), str(cl_b)])
plt.yticks([size / 2, size + pad + size / 2], [str(cl_b), str(cl_a)])
plt.plot([size + pad / 2, size + pad / 2], [0, 2 * size + pad], "k:")
plt.plot([0, 2 * size + pad], [size + pad / 2, size + pad / 2], "k:")
plt.axis([0, 2 * size + pad, 0, 2 * size + pad])
plt.xlabel("Predicted label")
plt.ylabel("True label")
save_fig("error_analysis_digits_plot")
plt.show()

this cell help us see if the given numbers are really weird that's why they where misunterpreted then we can do feature engeniiring like adding a feature that calculate loop closure

MULTI_label classification

Multi-label classification is the task where each sample can belong to zero, one, or multiple classes at the same time it's mostly used in tagging images , document categorization

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= '7')
y_train_odd = (y_train.astype('int8') % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

knn classifier needs if k petit =>of k grand => uf // tres sensible à la scale des featurs

In [ ]:
knn_clf.predict([some_digit])

In [ ]:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score(y_multilabel, y_train_knn_pred, average="macro")

In [ ]:
# extra code – shows that we get a negligible performance improvement when we
#              set average="weighted" because the classes are already pretty
#              well balanced.
f1_score(y_multilabel, y_train_knn_pred, average="weighted")

In [ ]:
from sklearn.multioutput import ClassifierChain

chain_clf = ClassifierChain(SVC(), cv=3, random_state=42)
chain_clf.fit(X_train[:2000], y_multilabel[:2000])
chain_clf.predict([some_digit])

Classifier Chains are a simple yet powerful extension of the One-vs-Rest approach that lets you model label correlations in multi-label problems.

# MULTIOUTPUT CLASSIFICATION

In [ ]:
np.random.seed(42)  # to make this code example reproducible
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise
y_train_mod = X_train
y_test_mod = X_test

In [ ]:
# extra code – this cell generates and saves Figure 3–12
plt.subplot(121); plot_digit(X_test_mod[0])
plt.subplot(122); plot_digit(y_test_mod[0])
save_fig("noisy_digit_example_plot")
plt.show()

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[0]])
plot_digit(clean_digit)
save_fig("cleaned_digit_example_plot")  # extra code – saves Figure 3–13
plt.show()

this approach isn't very good a multioutput regressor would be more natural here

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor(n_neighbors=5, weights="distance")
knn_reg.fit(X_train_mod, y_train_mod)           # y_train_mod = clean images
clean_digit = knn_reg.predict([X_test_mod[0]])  # shape (1,784)
plot_digit(clean_digit[0])
plt.show()


example of multi label

In [ ]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss, classification_report

# 1️⃣ Generate a random multi-label dataset
#    - n_samples=200 examples
#    - n_features=20 input features
#    - n_classes=3 possible labels per example
#    - each example has on average 2 labels
X, Y = make_multilabel_classification(
    n_samples=200,
    n_features=20,
    n_classes=3,
    n_labels=2,
    allow_unlabeled=False,
    random_state=42
)

# 2️⃣ Split into train / test
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

# 3️⃣ Wrap a base classifier for multi-output
#    Here: one binary LogisticRegression per label
base_clf = LogisticRegression(solver="liblinear")
multi_clf = OneVsRestClassifier(base_clf)

# 4️⃣ Train
multi_clf.fit(X_train, Y_train)

# 5️⃣ Predict on test set
Y_pred = multi_clf.predict(X_test)

# 6️⃣ Evaluate
print("Hamming loss:", hamming_loss(Y_test, Y_pred))
print("\nClassification report (per label):")
print(classification_report(Y_test, Y_pred, target_names=[
    "Label A", "Label B", "Label C"
]))
Y_pred[0]

categorical data + continous data

In [ ]:
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Suppose X_train is your features,
# Y_cont (n_samples×n_cont) continuous targets,
# Y_cat  (n_samples×n_cat) categorical targets.

# 1) Regression head
reg = MultiOutputRegressor(RandomForestRegressor())
reg.fit(X_train, Y_cont)
Y_cont_pred = reg.predict(X_test)

# 2) Classification head
clf = MultiOutputClassifier(RandomForestClassifier())
clf.fit(X_train, Y_cat)
Y_cat_pred = clf.predict(X_test)

# 3) Combine
import numpy as np
Y_pred = np.hstack([Y_cont_pred, Y_cat_pred])


exercice tunnig of hyperparametres

In [7]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [10]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn",   KNeighborsClassifier())
])
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9],
    "knn__weights":     ["uniform", "distance"],
    "knn__p":           [1, 2]            # 1=Manhattan, 2=Euclidean
}
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train, y_train)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 

évaluation des meilleur paramétres à l'aide de la cv et \de la méthode de gridsearch + l'usage du scaler avec le knn car le knn decide de la closeness avec des metrics de distance but be aware it recommends high computional power

## ⭐ whenever you consider using distance metrics think of scaling

In [ ]:
print("Best params:", grid.best_params_)
print("CV best accuracy: {:.3%}".format(grid.best_score_))

# 6️⃣ Evaluate on the held-out test set
y_pred = grid.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.3%}".format(test_acc))

IN orther to improve accuracy we can do a data augmentation by shifting the images

In [ ]:
from scipy.ndimage import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((-1, 0), (1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

code exemple for data augmentation

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

very importanted this code ungroup the shifted images

In [ ]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn",   KNeighborsClassifier(n_neighbors=5, weights="distance", p=1))
])
pipe.fit(X_train_aug, y_train_aug)

# 4️⃣ Evaluate on the original test set
y_pred = pipe.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Test set accuracy after data augmentation: {test_acc:.3%}")

data augmentation is a very interesting way to improve the accuracy